In [ ]:
import pandas as pd
import nltk as nk
import numpy as np

In [ ]:
metro_lyrics = pd.read_csv("/Users/purvank/Desktop/NLP IIT/Ipython_Notebooks/lyrics.csv")
song_lyrics = pd.read_csv("/Users/purvank/Desktop/NLP IIT/Ipython_Notebooks/Song_lyrics.csv")
del (metro_lyrics['year'])
del (song_lyrics['album'])
whole_data = metro_lyrics.append(song_lyrics)

In [ ]:
whole_data.dropna(inplace=True)

In [ ]:
whole_data.groupby('genre').count()

### Removing genres which have a low number of instances

In [ ]:
data = whole_data.drop(whole_data[whole_data.genre=='Other'].index)
data = data.drop(data[data.genre=='Not Available'].index)
data = data.drop(data[data.genre=='Jazz'].index)
data = data.drop(data[data.genre=='Folk'].index)
data = data.drop(data[data.genre=='R&B'].index)
data = data.drop(data[data.genre=='Electronic'].index)
data = data.drop(data[data.genre=='Country'].index)
data = data.drop(data[data.genre=='Indie'].index)

In [ ]:
data.groupby('genre').count()

In [ ]:
data.head()

### Remove stopwords and Stem text

In [ ]:
import nltk
from nltk.corpus import stopwords
with open('/Users/purvank/Desktop/NLP IIT/Ipython_Notebooks/stopwords.txt') as f:
    lines = f.read().splitlines()

In [ ]:
def StopStem(df):
    morestop = set(lines)
    stop = set(stopwords.words('english'))
    df['lyrics'] = df['lyrics'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    df['lyrics'] = df['lyrics'].apply(lambda x: ' '.join([word for word in x.split() if word not in (morestop)]))
    df["lyrics"] = df['lyrics'].str.replace('[^\w\s]','')
    return df

In [ ]:
songs = StopStem(data)

In [ ]:
songs.head()

### Fix Unicode issues in text :

In [ ]:
import ftfy
def fixUni(df):
    df['lyrics'] = df['lyrics'].apply(lambda x: ftfy.fix_text_encoding(x))
    #df['lyrics'] = df['lyrics'].apply(lambda x: x if ftfy.badness.sequence_weirdness(x)>0)
    return df

In [ ]:
songs = fixUni(data)

In [ ]:
songs.shape

In [ ]:
songs.shape

### Removing instances which have a non-english words in lyrics

In [ ]:
for index,row in songs.lyrics.iteritems():
    if(ftfy.badness.sequence_weirdness(row)<0):
        songs.lyrics[index] = np.NaN

In [ ]:
songs.dropna(inplace=True)

In [ ]:
songs.shape

### Removing parenthesis and brackets and removing unimportant words using regular expression.

In [ ]:
def remove_brackets(text):
    new_text = re.sub(r'\(.*?\)','',text)
    new_text = re.sub(r'\[.*?\]','',text)
    return new_text

In [ ]:
import re
def ReBr(df):
        df['lyrics'] = df['lyrics'].apply(remove_brackets)
        return df

In [ ]:
def Chorus(df):
    words = ['Chorus','Verse1','Verse2','Verse']
    df['lyrics'] = df['lyrics'].apply(lambda x: ' '.join([word for word in x.split() if word not in words]))
    return df

In [ ]:
songs = ReBr(songs)

In [ ]:
songs = Chorus(songs)

In [ ]:
songs.to_csv("ReallyReallyCleaned.csv")